In [4]:
import sys
sys.path.append('../..')
from explainn import tools
from explainn import networks
from explainn import train
from explainn import test
from explainn import interpretation
from explainn import utils
import torch
import os
from torch import nn
from sklearn.metrics import average_precision_score
from sklearn import metrics
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pandas as pd
import numpy as np
from explainn.train.train import train_explainn
from explainn.utils.tools import dna_one_hot
from explainn.models.networks import ExplaiNN


In [5]:

# Load the tsv file
df = pd.read_csv('C:/Users/aslak/Master/Github_first/SCRATCH/AS-TAC/AS-TAC_1000bp.tsv', sep='\t', header=None)


In [23]:
# print the head of the df second column
print(df[2:].head())
print(f"Shape of the df: {df.shape}")



                0                                                  1   2   3   \
2  1:385601-386600  GGGCATGTTGAGATAAATGTACTAACCGGGAGGGTTGAATGATGTA...   0   0   
3  1:386201-387200  GATTGGATAACTAGGATGCTGCTGGTAGAGTAGCTAGCTAGCTTAC...   0   0   
4  1:386401-387400  GATGCTGCTGGTAGAGTAGCTAGCTAGCTTCCTAGCTGTACCGACT...   0   0   
5  1:386601-387600  TTACCTAGCTGTACCGACTAGCTAGGATAACTAGGGTGCTGCTGGT...   0   0   
6  1:395601-396600  AGTTATTGATGATCTATTTCGTTGTCTTATGTTTATGTCCTTTTAA...   0   1   

   4   5   6   7   8   9   ...  55  56  57  58  59  60  61  62  63  64  
2   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  
3   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  
4   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  
5   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   0   0   0  
6   0   0   0   1   0   0  ...   0   0   1   0   0   1   0   0   0   0  

[5 rows x 65 columns]
Shape of the df: (1638680, 65)


In [63]:

# count the number of sequences containing at least one 'N'
count_N = df[1].str.contains('A').sum()

# count the number of sequences not of length 1000
count_not_1000 = (df[1].str.len() < 1000).sum()
good_lines = (df[1].str.len() == 1000) & (~df[1].str.contains('N'))
print(f"Number of sequences that contain at least one 'N': {count_N}")
print(f"Number of sequences that are not of length 1000: {count_not_1000}")
print(f"Number of sequences that are good: {good_lines.sum()}")
print(f"Fraction of sequences that are good: {good_lines.sum()/df.shape[0]}")



Number of sequences that contain at least one 'N': 1638603
Number of sequences that are not of length 1000: 0
Number of sequences that are good: 1638680
Fraction of sequences that are good: 1.0


In [64]:
#print the sequences that does not contain an 'A'
print(df[1][~df[1].str.contains('A')])

3981       TCTCTGTCTGTCTCGCTCTCTGTCTGTCTGTCTCGCTCTCTGTCTG...
286411     TCTCTCTCTCTGTGTCTCTCTCTCTCTGTGTCTCTGTGTGTGTCTC...
747627     CTGTCTCTGTTTCTCTGTCTGTCTCTGTCTGTCTCTGTTTCTCTGT...
1525743    TCGCTCTGCGTGTGGCGCTCTCTCGCTCTGCGTGTGGCGCTCTCGC...
1525744    CTCTGCGTGCGTGTGGCGCTCTCTCTCTCGCTCTGCGTGCGTGTGG...
                                 ...                        
1627339    TGTGTGTGTGTGTGTGTGTGTGTGTTGTCGGTGTGTGTGTGTGTGT...
1627340    GTGTGTGTGTTGTCGGTGTGTGTGTGTGTGTGTGTTGTCGGTGTGT...
1627341    GGTGTGTGTGTGTGTGTGTGTTGTCGGTGTGTGTGTGTGTGTGTTG...
1631206    TCTCTCTCTGTGTCTCTCTCTCTCTCTGTGTGTCTCTCTCTCTGTC...
1631231    CTCTCTGTGTGTGTGTGTGTCTGTCTGTCTCTCTCTCTGTGTGTGT...
Name: 1, Length: 77, dtype: object


In [29]:
# print rownames of test_feat
print(df[0])


0                          1:374401-375400
1                          1:374601-375600
2                          1:385601-386600
3                          1:386201-387200
4                          1:386401-387400
                        ...               
1638675    CAJNNT020004222.1:229401-230400
1638676    CAJNNT020004222.1:229601-230600
1638677    CAJNNT020004222.1:229801-230800
1638678    CAJNNT020004222.1:230001-231000
1638679    CAJNNT020004222.1:230201-231200
Name: 0, Length: 1638680, dtype: object


0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: 12, dtype: int64

In [25]:
# Let's make a .h5
import pandas as pd
import numpy as np
import h5py
from sklearn.model_selection import train_test_split

# make a subset of df with only 100 rows
df = df.iloc[:100, :]

# load target_labels
with open("../../dna-sequence-models/downloads/bed_list.txt", 'r') as f:
    labels = f.read().splitlines()

# Remove "AS-TAC-peaks/AtlanticSalmon_ATAC_" and ".mLb.clN_peaks.narrowPeak" from the strings in labels list
labels = [label.replace("AS-TAC-peaks/AtlanticSalmon_ATAC_", "").replace(".mLb.clN_peaks.narrowPeak", "") for label in labels]

# remove chrom ranges if there are 65 columns
if df.shape[1] == 65:
    df = df.drop(0, axis=1)
# separate sequences and binary features
seqs = df.iloc[:, 0]
features = df.iloc[:, 1:]

# one hot encode sequences
seqs_one_hot = np.array([dna_one_hot(str(seq)) for seq in seqs])
seqs = seqs.apply(lambda x: pd.Series(list(x)))

# split data into train, test, valid
train_seq, test_seq, train_feat, test_feat = train_test_split(seqs_one_hot, features, test_size=0.15, random_state=42)
train_seq, valid_seq, train_feat, valid_feat = train_test_split(train_seq, train_feat, test_size=0.05, random_state=42)


In [26]:

# create .h5 file
with h5py.File('AS-TAC.h5', 'w') as hf:
    hf.create_dataset('train_in', data=train_seq)
    hf.create_dataset('valid_in', data=valid_seq)
    hf.create_dataset('test_in', data=test_seq)
    hf.create_dataset('train_out', data=train_feat)
    hf.create_dataset('valid_out', data=valid_feat)
    hf.create_dataset('test_out', data=test_feat)
    hf.create_dataset('target_labels', data=labels)